In [20]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os
from azure.storage.blob import ContainerClient
import io
import time
from PIL import Image
import datetime

In [21]:
file_path_color_left = "data/00\image_left/"
file_path_color_right = "data/00\image_right/"

left_images = os.listdir(file_path_color_left)
right_images = os.listdir(file_path_color_right)

print(len(left_images))
# plt.figure(figsize=(12,4))
# plt.imshow(cv2.imread(file_path_color_left + left_images[165],0)) 
#turning it into greyscale because we don't really need color unless we're interested in matching moving objects/features. 

523


In [22]:
def pixel_coord_np(height,width):
    """
    Pixel in homogenous coordinate
    Returns:
        Pixel coordinate:       [3, width * height]
    """
    x = np.linspace(0, width - 1, width).astype(np.float32)
    y = np.linspace(0, height - 1, height).astype(np.float32)
    [x, y] = np.meshgrid(x, y)
    return np.vstack((x.flatten(), y.flatten(), np.ones_like(x.flatten())))


def intrinsic_from_fov(height, width, fov=90):
    """
    Basic Pinhole Camera Model
    intrinsic params from fov and sensor width and height in pixels
    Returns:
        K:      [4, 4]
    """
    px, py = (width / 2, height / 2)
    hfov = fov / 360. * 2. * np.pi
    fx = width / (2. * np.tan(hfov / 2.))

    vfov = 2. * np.arctan(np.tan(hfov / 2) * height / width)
    fy = height / (2. * np.tan(vfov / 2.))

    return np.array([[fx, 0, px, 0.],
                     [0, fy, py, 0.],
                     [0, 0, 1., 0.],
                     [0., 0., 0., 1.]])

def quaternion_rotation_matrix(Q):
    """
    Covert a quaternion into a full projection matrix.
 
    Input
    :param Q: A 7 element array representing translation and the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x4 element matrix representing the full projection matrix. 
             This projection matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """
    # Extract the values from Q
    t0 = Q[0]
    t1 = Q[1]
    t2 = Q[2]
    q0 = Q[3]
    q1 = Q[4]
    q2 = Q[5]
    q3 = Q[6]
     
    # First row of the rotation matrix
    r00 = 2 * (q0 * q0 + q1 * q1) - 1
    r01 = 2 * (q1 * q2 - q0 * q3)
    r02 = 2 * (q1 * q3 + q0 * q2)
     
    # Second row of the rotation matrix
    r10 = 2 * (q1 * q2 + q0 * q3)
    r11 = 2 * (q0 * q0 + q2 * q2) - 1
    r12 = 2 * (q2 * q3 - q0 * q1)
     
    # Third row of the rotation matrix
    r20 = 2 * (q1 * q3 - q0 * q2)
    r21 = 2 * (q2 * q3 + q0 * q1)
    r22 = 2 * (q0 * q0 + q3 * q3) - 1
     
    # 3x4 projection matrix
    pro_matrix = np.array([[r00, r01, r02, t0],
                           [r10, r11, r12, t1],
                           [r20, r21, r22, t2]])
                            
    return pro_matrix

In [23]:
class Dataset_Handler():
    def __init__(self, sequence, lidar=False, progress_bar=True, low_memory=True):
        
        
        # This will tell our odometry function if handler contains lidar info
        self.lidar = lidar
        # This will tell odometry functin how to access data from this object
        self.low_memory = low_memory
        
        # Set file paths and get ground truth poses
        self.seq_dir = "data\{}/".format(sequence)
        self.poses_dir = "data\{}.csv".format(sequence)
        self.depth_dir = "data\{}/".format(sequence)

        poses = pd.read_csv(self.poses_dir, header=None)
        
        # Get names of files to iterate through
        self.left_image_files = os.listdir(self.seq_dir + 'image_left')
        self.right_image_files = os.listdir(self.seq_dir + 'image_right')
        self.left_depth_files = os.listdir(self.depth_dir + 'depth_left')
        self.right_depth_files = os.listdir(self.depth_dir + 'depth_right')
        
        # self.velodyne_files = os.listdir(self.seq_dir + 'flow')
        self.num_frames = len(self.left_image_files)
        print(self.num_frames)
        # self.lidar_path = self.seq_dir + 'flow/'
        self.first_image_left = cv2.imread(self.seq_dir + 'image_left/' 
                                               + self.left_image_files[0])
        height = 480
        width = 640
        K = intrinsic_from_fov(height,width)
        self.P0 = np.array(K[:3,:4])
        self.P1 = np.array(K[:3,:4])
        
        
        # Get calibration details for scene
        # calib = pd.read_csv(self.seq_dir + 'calib.txt', delimiter=' ', header=None, index_col=0)
        # self.P0 = np.array(calib.loc['P0:']).reshape((3,4))
        # self.P1 = np.array(calib.loc['P1:']).reshape((3,4))
        # self.P2 = np.array(calib.loc['P2:']).reshape((3,4)) #RGB cams
        # self.P3 = np.array(calib.loc['P3:']).reshape((3,4)) #RGB cams
        # This is the transformation matrix for LIDAR
        # self.Tr = np.array(calib.loc['Tr:']).reshape((3,4))
        
        # Get times and ground truth poses
        self.times = np.array(pd.read_csv(self.seq_dir + 'times.txt', 
                                          delimiter=' ', 
                                          header=None))
        self.gt = np.zeros((len(poses), 3, 4))
        for i in range(len(poses)):
            self.gt[i] = np.array(quaternion_rotation_matrix(poses.iloc[i])).reshape((3, 4))
        
        # Get images and lidar loaded
        if self.low_memory:
            # Will use generators to provide data sequentially to save RAM
            # Use class method to set up generators
            self.reset_frames()
            # Store original frame to memory for testing functions
            self.first_image_left = cv2.imread(self.seq_dir + 'image_left/' 
                                               + self.left_image_files[0])
            self.first_image_right = cv2.imread(self.seq_dir + 'image_right/' 
                                               + self.right_image_files[0])
            self.second_image_left = cv2.imread(self.seq_dir + 'image_left/' 
                                               + self.left_image_files[1])
            self.first_depth_left = np.load(self.depth_dir + 'depth_left/'
                                                + self.left_depth_files[0])
            self.first_depth_right = np.load(self.depth_dir + 'depth_right/'
                                                + self.right_depth_files[0])
            self.second_depth_left = np.load(self.depth_dir + 'depth_left/'
                                                + self.left_depth_files[1])
                                                
            if self.lidar:
                self.first_pointcloud = np.fromfile(self.lidar_path + self.velodyne_files[0],
                                                    dtype=np.float32, 
                                                    count=-1).reshape((-1, 4))
            self.imheight = height
            self.imwidth = width
            
        else:
            # If RAM is not a concern (>32GB), pass low_memory=False
            if progress_bar:
                import progressbar
                bar = progressbar.ProgressBar(max_value=self.num_frames)
            self.images_left = []
            self.images_right = []
            self.depths_left = []
            self.depths_right =[]
            self.pointclouds = []
            for i, name_left in enumerate(self.left_image_files):
                name_right = self.right_image_files[i]
                d_left = self.left_depth_files
                d_right = self.right_depth_files
                self.images_left.append(cv2.imread(self.seq_dir + 'image_left/' + name_left))
                self.images_right.append(cv2.imread(self.seq_dir + 'image_right/' + name_right))
                self.depths_left.append(np.load(self.depth_dir + 'depth_left/' + d_left))
                self.depths_right.append(np.load(self.depth_dir + 'depth_right/' + d_right))
                if self.lidar:
                    pointcloud = np.fromfile(self.lidar_path + self.velodyne_files[i], 
                                             dtype=np.float32, 
                                             count=-1).reshape([-1,4])
                    self.pointclouds.append(pointcloud)
                if progress_bar:
                    bar.update(i+1)
                
            self.imheight = self.images_left[0].shape[0]
            self.imwidth = self.images_left[0].shape[1]
            # Keep consistent instance variable names as when using low_memory
            self.first_image_left = self.images_left[0]
            self.first_image_right = self.images_right[0]
            self.second_image_left = self.images_left[1]
            if self.lidar:
                self.first_pointcloud = self.pointclouds[0]
            
    def reset_frames(self):
        # Resets all generators to the first frame of the sequence
        self.images_left = (cv2.imread(self.seq_dir + 'image_left/' + name_left)
                            for name_left in self.left_image_files)
        self.images_right = (cv2.imread(self.seq_dir + 'image_right/' + name_right)
                            for name_right in self.right_image_files)
        self.depths_left = (np.load(self.depth_dir+'depth_left/' + d_left)
                            for d_left in self.left_depth_files)
        self.depths_right = (np.load(self.depth_dir+'depth_right/' + d_right)
                            for d_right in self.right_depth_files)                    
        if self.lidar:
            self.pointclouds = (np.fromfile(self.lidar_path + velodyne_file, 
                                            dtype=np.float32, 
                                            count=-1).reshape((-1, 4))
                                for velodyne_file in self.velodyne_files)
        pass
handler = Dataset_Handler("00")



523


In [24]:
def compute_left_disparity_map(left_image, right_image, matcher = 'bm', rgb = True, verbose = True):
    '''
    Takes a left and right pair of images to computes the disparity map for the left
    image. 
    
    Arguments:
    img_left -- image from left camera
    img_right -- image from right camera
    
    Optional Arguments:
    matcher -- (str) can be 'bm' for StereoBM or 'sgbm' for StereoSGBM matching.
    rgb -- (bool) set to True if passing RGB images as input. 
    verbose -- (bool) set to True to report matching type and time to compute
    
    Returns:
    disp_left -- disparity map for the left camera image
    
    '''
    sad_window = 6
    num_disparities = sad_window * 16
    block_size = 11
    matcher_name = matcher

    if matcher_name == 'bm':
        matcher = cv2.StereoBM_create(numDisparities=num_disparities,
                                        blockSize=block_size)
                                        
    elif matcher_name == 'sgbm':
        matcher = cv2.StereoSGBM_create(numDisparities=num_disparities,
                                        blockSize=block_size,
                                        P1 = 8*3*sad_window**2,
                                        P2 = 32*3*sad_window**2,
                                        mode = cv2.STEREO_SGBM_MODE_SGBM_3WAY)
    if rgb:
        left_image = cv2.cvtColor(left_image, cv2.COLOR_BGR2GRAY)
        right_image = cv2.cvtColor(right_image, cv2.COLOR_BGR2GRAY)
    
    start = datetime.datetime.now()
    disp_left = matcher.compute(left_image, right_image).astype(np.float32)/16
    end = datetime.datetime.now()

    if verbose:
        print(f'Time to compute disparity map using Stereo{matcher_name.upper()}:', end-start)
    
    return disp_left


In [25]:
first_left = cv2.imread(file_path_color_left + left_images[167])
first_right = cv2.imread(file_path_color_right + right_images[167])

disp = compute_left_disparity_map(left_image=first_left, right_image=first_right,
                                    matcher = 'bm', verbose = True)

plt.figure(figsize=(15,15))
plt.imshow(disp);

Time to compute disparity map using StereoBM: 0:00:00.009997


In [26]:
def decompose_projection_matrix(p):
    '''
    Shortcut to use cv2.decomposeProjectionMatrix(), which only returns k, r, t, and divides
    t by the scale, then returns it as a vector with shape (3,) (non-homogeneous)
    
    Arguments:
    p -- projection matrix to be decomposed
    
    Returns:
    k, r, t -- intrinsic matrix, rotation matrix, and 3D translation vector
    
    '''
    k, r, t, _, _, _, _ = cv2.decomposeProjectionMatrix(p)
    t = (t / t[3])[:3]
    
    return k, r, t
def calc_depth_map(disp_left, k_left, t_left, t_right, rectified=True):
    '''
    Assuming we don't have access to the depth map...
    
    Calculate depth map using a disparity map, intrinsic camera matrix, and translation vectors
    from camera extrinsic matrices (to calculate baseline). Note that default behavior is for
    rectified projection matrix for right camera. If using a regular projection matrix, pass
    rectified=False to avoid issues.

    
    
    Arguments:
    disp_left -- disparity map of left camera
    k_left -- intrinsic matrix for left camera
    t_left -- translation vector for left camera
    t_right -- translation vector for right camera
    
    Optional Arguments:
    rectified -- (bool) set to False if t_right is not from rectified projection matrix
    
    Returns:
    depth_map -- calculated depth map for left camera
    
    '''
    # Get focal length of x axis for left camera
    f = k_left[0][0]
    
    # Calculate baseline of stereo pair
    if rectified:
        b = t_right[0] - t_left[0] 
    else:
        b = t_left[0] - t_right[0]
        
    # Avoid instability and division by zero
    disp_left[disp_left == 0.0] = 0.1
    disp_left[disp_left == -1.0] = 0.1
    
    # Make empty depth map then fill with depth
    depth_map = np.ones(disp_left.shape)
    depth_map = f * b / disp_left
    
    return depth_map

In [27]:
handler.first_depth_left

array([[3.1132812, 3.1132812, 3.1152344, ..., 3.9003906, 3.9023438,
        3.9042969],
       [3.1171875, 3.1191406, 3.1191406, ..., 3.9082031, 3.9101562,
        3.9121094],
       [3.1230469, 3.1230469, 3.125    , ..., 3.9160156, 3.9179688,
        3.9199219],
       ...,
       [2.0664062, 2.0664062, 2.0683594, ..., 3.1738281, 3.2265625,
        3.2285156],
       [2.0625   , 2.0644531, 2.0644531, ..., 3.1679688, 3.21875  ,
        3.2207031],
       [2.0585938, 2.0605469, 2.0605469, ..., 3.2070312, 3.2089844,
        3.2128906]], dtype=float32)

In [28]:
def pointcloud2image(pointcloud, imheight, imwidth, Tr, P0):
    '''
    ...
    Takes a pointcloud of shape Nx4 and projects it onto an image plane, first transforming
    the X, Y, Z coordinates of points to the camera frame with tranformation matrix Tr, then
    projecting them using camera projection matrix P0.
    
    Arguments:
    pointcloud -- array of shape Nx4 containing (X, Y, Z, reflectivity)
    imheight -- height (in pixels) of image plane
    imwidth -- width (in pixels) of image plane
    Tr -- 3x4 transformation matrix between lidar (X, Y, Z, 1) homogeneous and camera (X, Y, Z)
    P0 -- projection matrix of camera (should have identity transformation if Tr used)
    
    Returns:
    render -- a (imheight x imwidth) array containing depth (Z) information from lidar scan
    
    '''

    pointcloud = pointcloud[pointcloud[:, 0] > 0]
    

    pointcloud = np.hstack([pointcloud[:, :3], np.ones(pointcloud.shape[0]).reshape((-1,1))])
    
    # Transform pointcloud into camera coordinate frame
    cam_xyz = Tr.dot(pointcloud.T)
    
    # Ignore any points behind the camera (probably redundant but just in case)
    cam_xyz = cam_xyz[:, cam_xyz[2] > 0]
    
    # Extract the Z row which is the depth from camera
    depth = cam_xyz[2].copy()
    
    # Project coordinates in camera frame to flat plane at Z=1 by dividing by Z
    cam_xyz /= cam_xyz[2]
    
    # Add row of ones to make our 3D coordinates on plane homogeneous for dotting with P0
    cam_xyz = np.vstack([cam_xyz, np.ones(cam_xyz.shape[1])])
    
    # Get pixel coordinates of X, Y, Z points in camera coordinate frame
    projection = P0.dot(cam_xyz)
    #projection = (projection / projection[2])
    
    # Turn pixels into integers for indexing
    pixel_coordinates = np.round(projection.T, 0)[:, :2].astype('int')
    #pixel_coordinates = np.array(pixel_coordinates)
    
    # Limit pixel coordinates considered to those that fit on the image plane
    indices = np.where((pixel_coordinates[:, 0] < imwidth)
                       & (pixel_coordinates[:, 0] >= 0)
                       & (pixel_coordinates[:, 1] < imheight)
                       & (pixel_coordinates[:, 1] >= 0)
                      )
    pixel_coordinates = pixel_coordinates[indices]
    depth = depth[indices]
    
    # Establish empty render image, then fill with the depths of each point
    render = np.zeros((imheight, imwidth))
    for j, (u, v) in enumerate(pixel_coordinates):
        if u >= imwidth or u < 0:
            continue
        if v >= imheight or v < 0:
            continue
        render[v, u] = depth[j]
    # Fill zero values with large distance so they will be ignored. (Using same max value)


    
    return render

In [29]:
def stereo_2_depth(img_left, img_right, P0, P1, matcher='bm', rgb=True, verbose=False, 
                   rectified=True):
    '''
    Takes stereo pair of images and returns a depth map for the left camera. If your projection
    matrices are not rectified, set rectified=False.
    
    Arguments:
    img_left -- image of left camera
    img_right -- image of right camera
    P0 -- Projection matrix for the left camera
    P1 -- Projection matrix for the right camera
    
    Optional Arguments:
    matcher -- (str) can be 'bm' for StereoBM or 'sgbm' for StereoSGBM
    rgb -- (bool) set to True if images passed are RGB. Default is False
    verbose -- (bool) set to True to report computation time and method
    rectified -- (bool) set to False if P1 not rectified to P0. Default is True
    
    Returns:
    depth -- depth map for left camera
    
    '''
    # Compute disparity map
    disp = compute_left_disparity_map(img_left, 
                                      img_right, 
                                      matcher=matcher, 
                                      rgb=rgb, 
                                      verbose=verbose)
    # Decompose projection matrices
    k_left, r_left, t_left = decompose_projection_matrix(P0)
    k_right, r_right, t_right = decompose_projection_matrix(P1)
    # Calculate depth map for left camera
    depth = calc_depth_map(disp, k_left, t_left, t_right)
    
    return depth

In [30]:
def extract_features(image, detector='sift', mask=None):
    """
    Find keypoints and descriptors for the image

    Arguments:
    image -- a grayscale image

    Returns:
    kp -- list of the extracted keypoints (features) in an image
    des -- list of the keypoint descriptors in an image
    """
    if detector == 'sift':
        det = cv2.SIFT_create()
    elif detector == 'orb':
        det = cv2.ORB_create()
    elif detector == 'surf':
        det = cv2.xfeatures2d.SURF_create()
        
    kp, des = det.detectAndCompute(image, mask)
    
    return kp, des

In [31]:
def match_features(des1, des2, matching='BF', detector='sift', sort=True, k=2):
    """
    Match features from two images

    Arguments:
    des1 -- list of the keypoint descriptors in the first image
    des2 -- list of the keypoint descriptors in the second image
    matching -- (str) can be 'BF' for Brute Force or 'FLANN'
    detector -- (str) can be 'sift or 'orb'. Default is 'sift'
    sort -- (bool) whether to sort matches by distance. Default is True
    k -- (int) number of neighbors to match to each feature.

    Returns:
    matches -- list of matched features from two images. Each match[i] is k or less matches for 
               the same query descriptor
    """
    if matching == 'BF':
        if detector == 'sift':
            matcher = cv2.BFMatcher_create(cv2.NORM_L2, crossCheck=False)
        elif detector == 'orb':
            matcher = cv2.BFMatcher_create(cv2.NORM_HAMMING2, crossCheck=False)
        matches = matcher.knnMatch(des1, des2, k=k)
    elif matching == 'FLANN':
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees=5)
        search_params = dict(checks=50)
        matcher = cv2.FlannBasedMatcher(index_params, search_params)
        matches = matcher.knnMatch(des1, des2, k=k)
    
    if sort:
        matches = sorted(matches, key = lambda x:x[0].distance)

    return matches

In [32]:
def visualize_matches(image1, kp1, image2, kp2, match):
    """
    Visualize corresponding matches in two images

    Arguments:
    image1 -- the first image in a matched image pair
    kp1 -- list of the keypoints in the first image
    image2 -- the second image in a matched image pair
    kp2 -- list of the keypoints in the second image
    match -- list of matched features from the pair of images

    Returns:
    image_matches -- an image showing the corresponding matches on both image1 and image2 or None if you don't use this function
    """
    image_matches = cv2.drawMatches(image1, kp1, image2, kp2, match, None, flags=2)
    plt.figure(figsize=(16, 6), dpi=100)
    plt.imshow(image_matches)

In [33]:
def filter_matches_distance(matches, dist_threshold):
    """
    Filter matched features from two images by distance between the best matches

    Arguments:
    match -- list of matched features from two images
    dist_threshold -- maximum allowed relative distance between the best matches, (0.0, 1.0) 

    Returns:
    filtered_match -- list of good matches, satisfying the distance threshold
    """
    filtered_match = []
    for m, n in matches:
        if m.distance <= dist_threshold*n.distance:
            filtered_match.append(m)

    return filtered_match

In [34]:
def estimate_motion(match, kp1, kp2, k, depth1=None, max_depth=3000):
    """
    Estimate camera motion from a pair of subsequent image frames

    Arguments:
    match -- list of matched features from the pair of images
    kp1 -- list of the keypoints in the first image
    kp2 -- list of the keypoints in the second image
    k -- camera intrinsic calibration matrix 
    
    Optional arguments:
    depth1 -- Depth map of the first frame. Set to None to use Essential Matrix decomposition
    max_depth -- Threshold of depth to ignore matched features. 3000 is default

    Returns:
    rmat -- estimated 3x3 rotation matrix
    tvec -- estimated 3x1 translation vector
    image1_points -- matched feature pixel coordinates in the first image. 
                     image1_points[i] = [u, v] -> pixel coordinates of i-th match
    image2_points -- matched feature pixel coordinates in the second image. 
                     image2_points[i] = [u, v] -> pixel coordinates of i-th match
               
    """
    rmat = np.eye(3)
    tvec = np.zeros((3, 1))
    
    image1_points = np.float32([kp1[m.queryIdx].pt for m in match])
    image2_points = np.float32([kp2[m.trainIdx].pt for m in match])

   
    # if depth1 is not None:
    cx = k[0, 2]
    cy = k[1, 2]
    fx = k[0, 0]
    fy = k[1, 1]
    object_points = np.zeros((0, 3))
    delete = []
    
    # Extract depth information of query image at match points and build 3D positions
    for i, (u, v) in enumerate(image1_points):
        z = depth1[int(v), int(u)]
        

        if z > max_depth:
            delete.append(i)
            continue
            
        # Use arithmetic to extract x and y (faster than using inverse of k)
        x = z*(u-cx)/fx
        y = z*(v-cy)/fy

        object_points = np.vstack([object_points, np.array([x, y, z])])

        # Equivalent math with dot product w/ inverse of k matrix, but SLOWER (see Appendix A)
        #object_points = np.vstack([object_points, np.linalg.inv(k).dot(z*np.array([u, v, 1]))])

    image1_points = np.delete(image1_points, delete, 0)

    image2_points = np.delete(image2_points, delete, 0)

    
    # Use PnP algorithm with RANSAC for robustness to outliers
    
    _,rvec, tvec, inliers = cv2.solvePnPRansac(object_points, image2_points, cameraMatrix=k, distCoeffs=None)

    rmat = cv2.Rodrigues(rvec)[0]

    
    return rmat, tvec, image1_points, image2_points

In [35]:
image_left11 = handler.first_image_left
image_plus11 = handler.second_image_left
depth_left11 = np.load(handler.depth_dir + 'depth_left/'
                                                + handler.left_depth_files[133])
image_right11 = handler.first_image_right
depth_plus11 = np.load(handler.depth_dir + 'depth_left/'
                                                + handler.left_depth_files[1])

kp0, des0 = extract_features(image_left11, 'sift')
kp1, des1 = extract_features(image_plus11, 'sift')
matches = match_features(des0, des1, matching='BF', detector='sift', sort=True)
print('Number of matches before filtering:', len(matches))
matches = filter_matches_distance(matches, 0.45)
print('Number of matches after filtering:', len(matches))
visualize_matches(image_left11, kp0, image_plus11, kp1, matches)

Number of matches before filtering: 47
Number of matches after filtering: 5


In [36]:
stereo_l = handler.images_left
stereo_r = handler.images_right

depth_l = handler.depths_left
depth_r = handler.depths_right

%matplotlib tk

xs = []
ys = []
zs = []
compute_times = []
fig1 = plt.figure()
##########################################
#uncomment this for viewing ground truth
################################################################################
ax = fig1.add_subplot(projection='3d')
ax.view_init(elev=-20, azim=270)
ax.plot(handler.gt[:,0,3], handler.gt[:,1,3], handler.gt[:,2,3], c = 'k')
###############################################################################
poses = (gt for gt in handler.gt)

for i in range(handler.num_frames):
    i
    img_l = next(stereo_l)
    img_r = next(stereo_r)
    D_left = next(depth_l)
    D_right = next(depth_r)


    start = datetime.datetime.now()
    disp = compute_left_disparity_map(img_l, img_r, matcher='bm')

    gt = next(poses)
    xs.append(gt[0,3])
    ys.append(gt[1,3])
    zs.append(gt[2,3])
    

    plt.plot(xs,ys, zs, c = 'chartreuse')
    plt.pause(0.00000000001)
     
    cv2.imshow('L-Camera', img_l)
    cv2.imshow('R-Camera', img_r)
    cv2.imshow('Disparity', disp)
    ##########################################
    #uncomment this for viewing depth image// BUT, super slow. 
    ################################################################################
    # plt.imshow(D_left)
    ################################################################################   
    plt.pause(0.00000000001)
    
    cv2.waitKey(1)

    end = datetime.datetime.now()
    compute_times.append(end-start)
plt.close()
cv2.destroyAllWindows()


Time to compute disparity map using StereoBM: 0:00:00.006027
Time to compute disparity map using StereoBM: 0:00:00.005975
Time to compute disparity map using StereoBM: 0:00:00.007059
Time to compute disparity map using StereoBM: 0:00:00.005005
Time to compute disparity map using StereoBM: 0:00:00.006033
Time to compute disparity map using StereoBM: 0:00:00.006032
Time to compute disparity map using StereoBM: 0:00:00.007002
Time to compute disparity map using StereoBM: 0:00:00.008971
Time to compute disparity map using StereoBM: 0:00:00.005008
Time to compute disparity map using StereoBM: 0:00:00.010032
Time to compute disparity map using StereoBM: 0:00:00.008002
Time to compute disparity map using StereoBM: 0:00:00.006968
Time to compute disparity map using StereoBM: 0:00:00.006027
Time to compute disparity map using StereoBM: 0:00:00.005998
Time to compute disparity map using StereoBM: 0:00:00.004999
Time to compute disparity map using StereoBM: 0:00:00.006000
Time to compute disparit

In [37]:

k,r,t,_,_,_,_ = cv2.decomposeProjectionMatrix(handler.P0)
depth = handler.first_depth_left
rmat, tvec, image1_points, image2_points = estimate_motion(matches,kp0,kp1,depth1 = depth,k=k)
transformation_matrix = np.hstack([rmat,tvec])


In [92]:
def visual_odometry(handler, detector='sift', matching='BF', filter_match_distance=None, 
                    stereo_matcher='bm', mask=None, depth_type='stereo', subset=None,
                    plot=False):
    '''
    Function to perform visual odometry on a sequence from the KITTI visual odometry dataset.
    Takes as input a Data_Handler object and optional parameters.
    
    Arguments:
    handler -- Data_Handler object instance
    
    Optional Arguments:
    detector -- (str) can be 'sift' or 'orb'. Default is 'sift'.
    matching -- (str) can be 'BF' for Brute Force or 'FLANN'. Default is 'BF'.
    filter_match_distance -- (float) value for ratio test on matched features. Default is None.
    stereo_matcher -- (str) can be 'bm' (faster) or 'sgbm' (more accurate). Default is 'bm'.
    mask -- (array) mask to reduce feature search area to where depth information available.
    depth_type -- (str) can be 'stereo' or set to None to use Essential matrix decomposition.
                        Note that scale will be incorrect with no depth used.
    subset -- (int) number of frames to compute. Defaults to None to compute all frames.
    plot -- (bool) whether to plot the estimated vs ground truth trajectory. Only works if
                   matplotlib is set to tk mode. Default is False.
    
    Returns:
    trajectory -- Array of shape Nx3x4 of estimated poses of vehicle for each computed frame.
    
    '''
    # Determine if handler has lidar data
    lidar = handler.lidar
    
    # Report methods being used to user
    print('Generating disparities with Stereo{}'.format(str.upper(stereo_matcher)))
    print('Detecting features with {} and matching with {}'.format(str.upper(detector), 
                                                                   matching))
    if filter_match_distance is not None:
        print('Filtering feature matches at threshold of {}*distance'.format(filter_match_distance))
    if lidar:
        print('Improving stereo depth estimation with lidar data')
    if subset is not None:
        #subset = subset + 1
        num_frames = subset
    else:
        # Set num_frames to one less than the number of frames so we have sequential images
        # in the last frame run.
        num_frames = handler.num_frames
        
    if plot:
        fig = plt.figure(figsize=(14, 14))
        ax = fig.add_subplot(projection='3d')
        ax.view_init(elev=-20, azim=270)
        xs = handler.gt[:, 0, 3]
        ys = handler.gt[:, 1, 3]
        zs = handler.gt[:, 2, 3]
        ax.set_box_aspect((np.ptp(xs), np.ptp(ys), np.ptp(zs)))
        ax.plot(xs, ys, zs, c='k')
    # Establish homogeneous transformation matrix. First pose is identity    
    T_tot = np.eye(4)
    trajectory = np.zeros((num_frames, 3, 4))
    trajectory[0] = T_tot[:3, :]
    imheight = handler.imheight
    imwidth = handler.imwidth




    # Decompose left camera projection matrix to get intrinsic k matrix
    k_left, r_left, t_left = decompose_projection_matrix(handler.P0)


    if handler.low_memory:
        handler.reset_frames()
        image_plus1 = next(handler.images_left)

    # Iterate through all frames of the sequence
    for i in range(num_frames - 1):
        # Stop if we've reached the second to last frame, since we need two sequential frames
        #if i == num_frames - 1:
        #    break
        # Start timer for frame
        start = datetime.datetime.now()
        # Get our stereo images for depth estimation
        if handler.low_memory:
            image_left = image_plus1
            image_right = next(handler.images_right)
            # Get next frame in the left camera for visual odometry
            image_plus1 = next(handler.images_left)
        else:
            image_left = handler.images_left[i]
            image_right = handler.images_right[i]
            # Get next frame in the left camera for visual odometry
            image_plus1 = handler.images_left[i+1]
        

        # Estimate depth if using stereo depth estimation (recommended)
        if depth_type == 'stereo':
            depth = stereo_2_depth(image_left, 
                                   image_right, 
                                   P0=handler.P0, 
                                   P1=handler.P1,
                                   matcher=stereo_matcher)
            # depth = (handler.first_depth_left)
        # Otherwise use Essential Matrix decomposition (ambiguous scale)
        else:
            depth = None
            
        # Supercede stereo depth estimations where lidar points are available
        if lidar:
            if handler.low_memory:
                pointcloud = next(handler.pointclouds)
            else:
                pointcloud = handler.pointclouds[i]
            lidar_depth = pointcloud2image(pointcloud, 
                                           imheight=imheight, 
                                           imwidth=imwidth, 
                                           Tr=handler.Tr, 
                                           P0=handler.P0)
            indices = np.where(lidar_depth > 0)
            depth[indices] = lidar_depth[indices]

        # Get keypoints and descriptors for left camera image of two sequential frames
        kp0, des0 = extract_features(image_left, detector, mask)
        kp1, des1 = extract_features(image_plus1, detector, mask)
        
        # Get matches between features detected in the two images
        matches_unfilt = match_features(des0, 
                                        des1, 
                                        matching=matching, 
                                        detector=detector, 
                                        sort=True)
        
        # Filter matches if a distance threshold is provided by user
        if filter_match_distance is not None:
            matches = filter_matches_distance(matches_unfilt, filter_match_distance)
        else:
            matches = matches_unfilt
            
        # Estimate motion between sequential images of the left camera

        rmat, tvec, img1_points, img2_points = estimate_motion(matches, kp0, kp1, k=k_left, depth1=depth)

        # Create blank homogeneous transformation matrix
        Tmat = np.eye(4)
        # Place resulting rotation matrix  and translation vector in their proper locations
        # in homogeneous T matrix
        Tmat[:3, :3] = rmat
        Tmat[:3, 3] = tvec.T

        T_tot = T_tot.dot(np.linalg.inv(Tmat))
            
        # Place pose estimate in i+1 to correspond to the second image, which we estimated for
        trajectory[i+1, :, :] = T_tot[:3, :]
        # End the timer for the frame and report frame rate to user
        end = datetime.datetime.now()
        print('Time to compute frame {}:'.format(i+1), end-start)
        
        if plot:
            xs = trajectory[:i+2, 0, 3]
            ys = trajectory[:i+2, 1, 3]
            zs = trajectory[:i+2, 2, 3]
            plt.plot(xs, ys, zs, c='chartreuse')
            plt.pause(1e-32)
            
    if plot:        
        plt.close()
        
    return trajectory

In [93]:
handler.lidar = False
start = datetime.datetime.now()
trajectory_nolidar_bm = visual_odometry(handler,
                                        filter_match_distance=0.4,
                                        detector='sift',
                                        # matching='FLANN',
                                        stereo_matcher='bm',
                                        mask=None,     
                                        subset=100,
                                        plot=True)
end = datetime.datetime.now()
print('Time to perform odometry:', end-start)

Generating disparities with StereoBM
Detecting features with SIFT and matching with BF
Filtering feature matches at threshold of 0.4*distance
Time to compute frame 1: 0:00:00.131964
Time to compute frame 2: 0:00:00.120015
Time to compute frame 3: 0:00:00.115006
Time to compute frame 4: 0:00:00.124001
Time to compute frame 5: 0:00:00.115001
Time to compute frame 6: 0:00:00.123993
Time to compute frame 7: 0:00:00.113998
Time to compute frame 8: 0:00:00.115041
Time to compute frame 9: 0:00:00.110005
Time to compute frame 10: 0:00:00.118999
Time to compute frame 11: 0:00:00.118939
Time to compute frame 12: 0:00:00.120991
Time to compute frame 13: 0:00:00.114034
Time to compute frame 14: 0:00:00.116001
Time to compute frame 15: 0:00:00.114000
Time to compute frame 16: 0:00:00.118031
Time to compute frame 17: 0:00:00.110000
Time to compute frame 18: 0:00:00.119963
Time to compute frame 19: 0:00:00.115973
Time to compute frame 20: 0:00:00.113968
Time to compute frame 21: 0:00:00.114004
Time t

In [88]:
%matplotlib notebook

In [89]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111, projection='3d')

ax.plot(trajectory_nolidar_bm[:, :, 3][:, 0], 
        trajectory_nolidar_bm[:, :, 3][:, 1], 
        trajectory_nolidar_bm[:, :, 3][:, 2], label='estimated', color='orange')

ax.plot(handler.gt[:, :, 3][:, 0], 
        handler.gt[:, :, 3][:, 1], 
        handler.gt[:, :, 3][:, 2], label='ground truth')

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

ax.view_init(elev=-20, azim=270)
